In [1]:
from models_utils.Datasets import *
from models_utils.GLOBALS import *
import torch.nn.functional as F
from CNN.CNN import MultivariateCNN
from CNN.cnn_utils import train_cnn, get_train_data

In [2]:
# get train data
train_data = pd.read_csv('train.csv', index_col=0)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [ ]:
# sizes of padding/cutting
target_size_type1 = 3000
target_size_type2 = 1169

In [ ]:
# train or load models
train_or_load_autoencoders = 'load'
if train_or_load_autoencoders == 'train':
    model_CNN_type1 = train_cnn('Type1OnlyCNN', data_type_1, '1', target_size_type1, 30, 64, 0.001)
    model_CNN_type2 = train_cnn('Type2OnlyCNN', data_type_2, '2', target_size_type2, 20, 64, 0.001)
elif train_or_load_autoencoders == 'load':
    model_CNN_type1 = MultivariateCNN(3, target_size_type1, 18).to(device)
    model_CNN_type1.load_state_dict(torch.load('Type1OnlyCNN.pth'))
    model_CNN_type2 = MultivariateCNN(3, target_size_type2, 18).to(device)
    model_CNN_type2.load_state_dict(torch.load(f'Type2OnlyCNN.pth'))
else:
    raise ValueError('Wrong train or load')

In [ ]:
# calculate or load train_data
embedding_names = [f'embedding_feature_{i + 1}' for i in range(18)]
calculate_or_load_train_data = 'load'
if calculate_or_load_train_data == 'calculate':
    data_type_1, data_type_2 = get_train_data(['train_Only_CNN'], model_CNN_type1, model_CNN_type2,
                                              embedding_size=18, is_autoencoder=False)
elif calculate_or_load_train_data == 'load':
    data_type_1 = pd.read_csv('train_Only_CNN_type1.csv')
    data_type_2 = pd.read_csv('train_Only_CNN_type2.csv')

In [39]:
# get and save results for test data
results_list = []
for i, file_id in enumerate(pd.read_csv('sample_submission.csv')['sample_id'].to_list()):

    class_path = os.path.join(files_directory, f"{file_id}.csv")
    new_data = pd.read_csv(class_path)

    if new_data.shape[1] == 3:
        if len(new_data) < 4000:
            new_data = pad_sequence(new_data, 4000)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)
        logits = model_CNN_type1(normalized_new_data)


    else:
        new_data = new_data[new_data.iloc[:, 0] == 'acceleration [m/s/s]'].iloc[:, 1:]
        if len(new_data) < 1350:
            new_data = pad_sequence(new_data, 1350)
        new_data = torch.tensor(new_data.values, dtype=torch.float32).to(device)
        normalized_new_data = (new_data - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
        normalized_new_data = normalized_new_data.transpose(0, 1)

        logits = model_CNN_type2(normalized_new_data)

    predictions = F.softmax(logits, dim=1)
    res_dict = {activity: predictions.squeeze()[id].item() for id, activity in id_activity_mapping.items()}

    result_dict = {label: res_dict.get(label, 0) for label in activity_id_mapping.keys()}
    result_dict['sample_id'] = file_id
    results_list.append(result_dict)
results = pd.DataFrame(results_list, columns=['sample_id'] + list(activity_id_mapping.keys()))
results.fillna(0).to_csv('results_raw_cnn.csv', index=False)